In [1]:
import pandas as pd
import numpy as np

# Load data
btc = pd.read_csv("btc_data.csv")
eth = pd.read_csv("eth_data.csv")
stocks = pd.read_csv("tech_stocks_data.csv")
events = pd.read_csv("event_dates.csv")

# Ubah kolom Date jadi datetime
btc['Date'] = pd.to_datetime(btc['Date'])
eth['Date'] = pd.to_datetime(eth['Date'])
stocks['Date'] = pd.to_datetime(stocks['Date'])
events['Date'] = pd.to_datetime(events['Date'])

# Cek data
btc.head()


,Date,timestamp,price
0,2025-01-12,1.736640e+12,94559.551672
1,2025-01-13,1.736726e+12,94454.770896
2,2025-01-14,1.736813e+12,94456.349375
3,2025-01-15,1.736899e+12,96574.077688
4,2025-01-16,1.736986e+12,100313.152398


In [2]:
# Gabungkan berdasarkan kolom 'Date'
merged = btc.rename(columns={"price": "BTC"})
merged = merged.merge(eth.rename(columns={"price": "ETH"}), on="Date", how="outer")
merged = merged.merge(stocks, on="Date", how="outer")

# Urutkan tanggal
merged = merged.sort_values("Date").reset_index(drop=True)

merged.head()


,Date,timestamp_x,BTC,timestamp_y,ETH,AAPL,TSLA,NVDA
0,2025-01-12,1.736640e+12,94559.551672,1.736640e+12,3283.591323,NaN,NaN,NaN
1,2025-01-13,1.736726e+12,94454.770896,1.736726e+12,3264.070969,233.83592224121094,403.30999755859375,133.20849609375
2,2025-01-14,1.736813e+12,94456.349375,1.736813e+12,3134.552039,232.71861267089844,396.3599853515625,131.7387237548828
3,2025-01-15,1.736899e+12,96574.077688,1.736899e+12,3224.800648,237.2975616455078,428.2200012207031,136.218017578125
4,2025-01-16,1.736986e+12,100313.152398,1.736986e+12,3447.034048,227.710693359375,413.82000732421875,133.54844665527344


In [4]:
# Daftar kolom harga
price_cols = ['BTC', 'ETH', 'AAPL', 'TSLA', 'NVDA']

# Konversi semua kolom harga ke float (jaga-jaga kalau masih berupa string)
for col in price_cols:
    merged[col] = pd.to_numeric(merged[col], errors='coerce')

# Hitung return harian (%)
for col in price_cols:
    merged[f'{col}_ret'] = merged[col].pct_change()


In [5]:
# Hitung rolling standard deviation (volatilitas) 3 hari untuk return harian
for col in price_cols:
    ret_col = f"{col}_ret"
    merged[f'{col}_vol'] = merged[ret_col].rolling(window=3).std()


In [6]:
merged[['Date', 'BTC', 'BTC_ret', 'BTC_vol', 'AAPL', 'AAPL_ret', 'AAPL_vol']].tail(10)


,Date,BTC,BTC_ret,BTC_vol,AAPL,AAPL_ret,AAPL_vol
171,2025-07-02,105613.399742,-0.014182,0.013249,212.440002,0.022231,0.004924
172,2025-07-03,108824.444232,0.030404,0.025045,213.550003,0.005225,0.008516
173,2025-07-04,109602.204839,0.007147,0.022300,NaN,0.000000,0.011624
174,2025-07-05,108040.891940,-0.014245,0.022331,NaN,0.000000,0.003017
175,2025-07-06,108217.468500,0.001634,0.011107,NaN,0.000000,0.000000
176,2025-07-07,109215.197718,0.009220,0.011974,209.949997,-0.016858,0.009733
177,2025-07-08,108300.716758,-0.008373,0.008824,210.009995,0.000286,0.009816
178,2025-07-09,108953.191877,0.006025,0.009372,211.139999,0.005381,0.011651
179,2025-07-10,111239.940476,0.020988,0.014682,NaN,0.000000,0.003027
180,NaT,NaN,0.000000,0.010807,NaN,0.000000,0.003107


In [7]:
# Buat kolom penanda event
merged['Event'] = np.where(merged['Date'].isin(events['Date']), 
                           merged['Date'].map(events.set_index('Date')['Event']), 
                           None)

# Tambah kolom is_event_day (boolean)
merged['is_event_day'] = merged['Event'].notna()

merged[merged['is_event_day']].head()


,Date,timestamp_x,BTC,timestamp_y,ETH,AAPL,TSLA,NVDA,BTC_ret,ETH_ret,AAPL_ret,TSLA_ret,NVDA_ret,BTC_vol,ETH_vol,AAPL_vol,TSLA_vol,NVDA_vol,Event,is_event_day
19,2025-01-31,1.738282e+12,104781.513510,1.738282e+12,3248.249484,235.432083,404.600006,120.050621,0.010244,0.042966,-0.006692,0.010792,-0.036743,0.015064,0.036942,0.006741,0.026042,0.026964,FOMC,True
32,2025-02-13,1.739405e+12,97836.188561,1.739405e+12,2736.270314,241.213684,355.940002,135.268158,0.021895,0.051185,0.019673,0.057740,0.031646,0.019816,0.036439,0.001794,0.062565,0.023783,CPI,True
59,2025-03-12,1.741738e+12,82799.108029,1.741738e+12,1921.309923,216.695831,248.089996,115.731964,0.050964,0.022450,-0.017479,0.075939,0.064276,0.057779,0.057174,0.015654,0.123410,0.057753,CPI,True
67,2025-03-20,1.742429e+12,86815.441095,1.742429e+12,2060.728601,213.819611,236.259995,118.521767,0.048749,0.066190,-0.005296,0.001696,0.008594,0.032084,0.032908,0.010226,0.050174,0.027920,FOMC,True
109,2025-05-01,1.746058e+12,94235.753310,1.746058e+12,1794.049541,213.040634,280.519989,111.602249,-0.000219,-0.001626,0.003859,-0.005812,0.024697,0.010693,0.002748,0.001126,0.027656,0.013870,FOMC,True


In [8]:
merged.to_csv("merged_data.csv", index=False)
print("✅ Data selesai diproses dan disimpan sebagai merged_data.csv")


✅ Data selesai diproses dan disimpan sebagai merged_data.csv


In [9]:
merged.loc[merged['is_event_day'], ['Date', 'Event', 'BTC', 'BTC_ret', 'AAPL', 'AAPL_ret']]


,Date,Event,BTC,BTC_ret,AAPL,AAPL_ret
19,2025-01-31,FOMC,104781.513510,0.010244,235.432083,-0.006692
32,2025-02-13,CPI,97836.188561,0.021895,241.213684,0.019673
59,2025-03-12,CPI,82799.108029,0.050964,216.695831,-0.017479
67,2025-03-20,FOMC,86815.441095,0.048749,213.819611,-0.005296
109,2025-05-01,FOMC,94235.753310,-0.000219,213.040634,0.003859
